In [ ]:
#ライブラリ、モジュールのインポート
import os 
import pandas as pd
import glob
from sklearn.model_selection import train_test_split


In [ ]:
#GPUの確認
import torch
if torch.cuda.is_available():
  print("GPUの設定ができています。")
else:
  print("GPUの設定ができていません！")

In [ ]:
自分で用意したデータの取得
# ディレクトリとCSVファイルのパス設定
data_path = '/home/yasumasa-tezuka/data_intern/images/'
csv_file = '/home/yasumasa-tezuka/data_intern/metadata.csv'

# CSVファイルの読み込み（ヘッダー行がないことを明示）
df = pd.read_csv(csv_file, header=None)

# 列名を手動で設定
df.columns = ['FileNumber', 'Name', 'Type', 'Location', 'Season', 'City', 'Distance', 'Visibility']

# ディレクトリ内の全ての画像ファイルを取得
image_filenames = glob.glob(os.path.join(data_path, '*.jpg'))

# 各画像ファイルに対してループしてテキストファイルを生成
for image_path in image_filenames:
    image_filename = os.path.basename(image_path)
    image_filename_without_extension = os.path.splitext(image_filename)[0]

    # 画像ファイル名を数値に変換
    image_number = int(image_filename_without_extension)

    # ファイル名がdf['FileNumber']列内に存在するかを確認
    if image_number in df['FileNumber'].values:
        row = df[df['FileNumber'] == image_number].iloc[0]

        # テキストファイルのパスを生成
        txt_filename = image_filename_without_extension + '.txt'
        txt_path = os.path.join(data_path, txt_filename)

        # CSVのデータをテキストファイルに書き込む
        with open(txt_path, 'w') as file:
            for col in df.columns:
                file.write(f"{col}: {row[col]}\n")

# 画像ファイルとテキストファイルのリストを作成
image_files = glob.glob(os.path.join(data_path, "*.jpg"))
text_files = [os.path.splitext(img)[0] + '.txt' for img in image_files]

# 存在するテキストファイルのみを保持
image_files = [img for img, txt in zip(image_files, text_files) if os.path.exists(txt)]
text_files = [txt for txt in text_files if os.path.exists(txt)]

# データを学習用とテスト用に分割
x_train, x_test, y_train, y_test = train_test_split(image_files, text_files, test_size=0.3, random_state=42)

# 分割されたデータのサイズを表示
print(f"学習データの画像数: {len(x_train)}, テキスト数: {len(y_train)}")
print(f"テストデータの画像数: {len(x_test)}, テキスト数: {len(y_test)}")


In [ ]:
#!pip install pyyamlライブラリのインストール

import yaml

file_path = '/home/yasumasa-tezuka/data_intern/annotation_images/data.yaml'

with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# データを更新
data['train'] = '/home/yasumasa-tezuka/data_intern/annotation_images/train/images'
data['val'] = '/home/yasumasa-tezuka/data_intern/annotation_images/valid/images'
data['test'] = '/home/yasumasa-tezuka/data_intern/annotation_images/test/images'
data['nc'] = 1
data['names'] = ['tree']

data['roboflow'] = {
    'workspace': 'yasumasa-tezuka',
    'project': 'tree-number',
    'version': 1,
    'license': 'CC BY 4.0',
    'url': 'https://universe.roboflow.com/yasumasa-tezuka/tree-number/dataset/1'
}

# 変更をファイルに保存
with open(file_path, 'w') as file:
    yaml.dump(data, file)